In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings

sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

In [2]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [3]:
word2idx = {}
idx2word = {}
with open('./data/w2id.txt', 'r') as f:
    for line in f.readlines():
        temp = line.split()
        word2idx[temp[0]] = temp[1]

In [4]:
word2idx['bạn']

'12703'

In [5]:
unigram = {}
with open('./data/1-gram.txt', 'r') as f:
    for line in f.readlines():
        temp = line.split()
        unigram[f'{temp[0]}'] = int(temp[1])

bigram = {}
with open('./data/2-gram.txt', 'r') as f:
    for line in f.readlines():
        temp = line.split()
        bigram[f'{temp[0]} {temp[1]}'] = int(temp[2])

trigram = {}
with open('./data/3-gram.txt', 'r') as f:
    for line in f.readlines():
        temp = line.split()
        trigram[f'{temp[0]} {temp[1]} {temp[2]}'] = int(temp[3])

In [6]:
bigram

{'7053 7148': 80,
 '228 4806': 634,
 '8185 11250': 139,
 '4558 10297': 1344,
 '5134 3303': 1040,
 '12917 916': 9236,
 '11360 15143': 70,
 '3442 1351': 78,
 '8420 12885': 225,
 '5187 6471': 101,
 '535 1254': 1053,
 '8361 3558': 74,
 '7032 7467': 605,
 '2351 15793': 1508,
 '13626 7438': 473,
 '9161 10036': 261,
 '15811 3565': 70,
 '9890 4232': 309,
 '11360 14061': 73,
 '13306 8540': 712,
 '6235 12070': 841,
 '10061 15643': 5493,
 '6087 10188': 78,
 '1213 4152': 269,
 '3105 4647': 113,
 '587 916': 179,
 '8361 4152': 180,
 '12712 7438': 110,
 '4242 6925': 55,
 '9890 7602': 60,
 '8361 7829': 55,
 '15770 3209': 985,
 '7969 14280': 195,
 '13697 2758': 358,
 '13797 5696': 161,
 '1141 3750': 75,
 '3482 4806': 59,
 '12313 1239': 8617,
 '11428 13565': 70,
 '8786 8766': 100,
 '2776 4061': 1189,
 '2929 14505': 452,
 '10001 14698': 129,
 '3572 12767': 52,
 '6256 1021': 1160,
 '8821 6397': 56,
 '228 8603': 195,
 '4485 14681': 77,
 '2776 5688': 421,
 '3245 253': 69,
 '1800 6839': 53,
 '5514 14391': 16

In [7]:
import json


class NGramGenerator:
    def __init__(self, unigram, bigram, trigram, word2idx):
        self.word2idx = word2idx
        self.total_words = sum(unigram.values())
        self.idx2word = {v: k for k, v in word2idx.items()}
        self.n_gram = {
            1: unigram,
            2: bigram,
            3: trigram
        }
        self.syllables = json.load(open('./data/syllables.json', 'r'))

    def prob_with_ngram_backoff(self, prob_word, sentences):
        n = len(sentences) + 1
        if n > 3:
            raise ValueError('Sentences should be less than 3 words')
        if prob_word not in self.word2idx:
            return 0
        previous_n_word_ids = [str(self.word2idx[word]) for word in sentences[-n+1:]]
        pred_word_id = str(self.word2idx[prob_word])
        
        for i in range(n-1, 0, -1):
            key_numerator = ' '.join(previous_n_word_ids[-i:] + [pred_word_id])
            key_denominator = ' '.join(previous_n_word_ids[-i:])
            if key_numerator in self.n_gram[i+1] and key_denominator in self.n_gram[i]:
                return self.n_gram[i+1][key_numerator] / self.n_gram[i][key_denominator]
        if pred_word_id in self.n_gram[1]:
            return self.n_gram[1][pred_word_id] / self.total_words
        return 0
    
    def prob_with_ngram_interpolation(self, prob_word, sentences, lambda_1=0.1, lambda_2=0.3, lambda_3=0.6):
        n = len(sentences) + 1
        if n > 3:
            raise ValueError('Sentences should be less than 3 words')
        if prob_word not in self.word2idx:
            return 0
        previous_n_word_ids = [str(self.word2idx[word]) for word in sentences[-n+1:]]
        prob_word_id = str(self.word2idx[prob_word])

        l = [lambda_1, lambda_2, lambda_3]

        total_prob, total_lambda = 0, 0
        for i in range(n-1, 0, -1):
            key_numerator = ' '.join(previous_n_word_ids[-i:] + [prob_word_id])
            key_denominator = ' '.join(previous_n_word_ids[-i:])
            if key_numerator in self.n_gram[i+1] and key_denominator in self.n_gram[i]:
                total_prob += l[i] * self.n_gram[i+1][key_numerator] / self.n_gram[i][key_denominator]
                total_lambda += l[i]
        if prob_word_id in self.n_gram[1]:
            total_prob += l[0] * self.n_gram[1][prob_word_id] / self.total_words
            total_lambda += l[0]
        
        if total_lambda == 0:
            return 0

        return total_prob / total_lambda
    
    def add_next_word(self, sentences, n_prev=2, method = 'backoff'):
        prob_func = self.prob_with_ngram_backoff if method == 'backoff' else self.prob_with_ngram_interpolation
        if n_prev >= 3:
            raise ValueError('n_prev should be less than 3')
        prob = {}
        for word in self.word2idx:
            prob[word] = prob_func(word, sentences)
        prob = sorted(prob.items(), key=lambda x: x[1], reverse=True)
        sentences.append(prob[0][0])
        return sentences
    
    def generate_sentence(self, sentences, n_prev=2, n_next=50, method='backoff'):
        for i in range(n_next):
            sentences = self.add_next_word(sentences[-n_prev:], n_prev, method)
            print(sentences[-1])

    def _beam_search(self, syllables:dict[str,list[str]], sentence:list[str], beam_size:int):
        result_sentences:list[tuple[list[str],float]] = [] # ex: [(['xin', 'chào'], 0.5), (['xin', 'chảo'], 0.3)]
        for _ in range(beam_size):
            result_sentences.append(([], 0))

        # main beam search
        for removed_accent_word in sentence:
            prob_results = {}
            for tp in result_sentences:
                for word in syllables[removed_accent_word]:

                    # CHECK:
                    if word not in self.word2idx:
                        continue

                    temp_sentence = tp[0].copy()
                    temp_sentence.append(word)
                    prob_results[tuple(temp_sentence)] = \
                        tp[1] + np.log(self.prob_with_ngram_interpolation(word, temp_sentence[-3:-1]) + 1e-9)
            sorted_prob_results = sorted(prob_results.items(), key=lambda x: x[1], reverse=True)
            result_sentences = [(list(item[0]), item[1]) for item in sorted_prob_results[:beam_size]]
        
        return ' '.join(result_sentences[0][0]) # return the best sentence
    
    def fix_accent(self, sentence, beam_size=5):
        sentence = sentence.split()
        return self._beam_search(self.syllables, sentence, beam_size)
    
    def fix_accent_and_generate_sentence(self, sentence, n_next=50):
        fixed_sentence = self.fix_accent(sentence, 30)
        print(fixed_sentence)
        self.generate_sentence(fixed_sentence.split(), n_next=n_next)

In [8]:
generator = NGramGenerator(unigram, bigram, trigram, word2idx)

In [9]:
generator.fix_accent('hom nay toi muon di choi cong vien voi con cho cua toi nhung thoi tiet khong tot lam vay nen toi o nha choi voi con meo cua me toi o trong phong khach')

'hôm nay tôi muốn đi chơi công viên với con chó của tôi nhưng thời tiết không tốt làm vậy nên tôi ở nhà chơi với con mèo của mẹ tôi ở trong phòng khách'

In [10]:
generator.fix_accent('hom qua troi mua lon va toi khong mang theo ao mua nen toi phai o lai cong ty de cho het mua roi moi di bo ve nha')

'hôm qua trời mưa lớn và tôi không mang theo áo mưa nên tôi phải ở lại công ty để cho hết mùa rồi mới đi bộ về nhà'

In [11]:
sent = 'ngay mai du bao thoi tiet troi nang the nen toi co du dinh se di tham quan cong vien voi ban be bang chiec xe dap moi mua vao ngay hom kia'
generator.fix_accent(sent)

'ngày mai dự báo thời tiết trời nắng thế nên tôi có dự định sẽ đi thăm quan công viên với bạn bè bằng chiếc xe đạp mới mua vào ngày hôm kia'

In [12]:
generator.fix_accent_and_generate_sentence('toi muon mua mot chiec xe o to de tien cho viec di lai trong noi thanh', n_next=30)

tôi muốn mua một chiếc xe ô tô để tiện cho việc đi lại trong nội thành
hà
nội
dung
lượng
numberrr
mah
và
chạy
hệ
điều
hành
vi
của
mình
và
không
có
mắc
không
có
mắc
không
có
mắc
không
có
mắc
không
có
mắc


cần
